## 데이터 전처리

- 한글만 남길 때 정규 표현식 참고: https://haah.kr/2017/08/23/alphabetic-letter-validation/

- 불용어 리스트: https://github.com/konlpy/konlpy/pull/310

- `TypeError: expected string or bytes-like object, got 'NoneType'` 에러 발생 = 리뷰 텍스트 중 None 값이 있음 -> 전처리 필요

In [2]:
import pandas as pd

In [3]:
df = pd.read_parquet('output.parquet')

In [3]:
df = pd.read_parquet('./csv_files/steam_reviews_HFF_flatten.parquet')

# 영어 제거

In [1]:
import re

def detect_all_eng(text):
    if not isinstance(text, str):
        return ''
    
    text = re.sub(r"[0-9\s]*", "", text)
    
    if re.fullmatch(r"[A-Za-z\s]*", text):
        return ''
    else:
        return text

In [ ]:
df['review'] = df['review'].apply(detect_all_eng)

In [10]:
df.shape

(31250, 21)

## None Review Data 삭제, URL/특정 이모지 제외 특수문자 등/공백 제거

In [11]:
import re

def preprocess_text(text):
    if text is None:
        return ''
    
    # URL 제거 (http 또는 www로 시작하는 URL 제거)
    text = re.sub(r'(http\S+|www\.\S+)', '', text) 

    # 특정 이모지와 한글, 숫자, 공백, 밑줄 이외의 문자 제거
    text = re.sub(r'[^\w\s\u2705\u2611\u2714\u26D4\u1F6AB\u2716\u274C\u274E\u1F534\u1F7E2가-힣]', '', text)

    # 한글 자음, 모음 제거
    text = re.sub(r'\b[ㄱ-ㅎㅏ-ㅣ]+\b', '', text)

    # 중국어 및 일본어 문자 제거
    text = re.sub(r'[\u4E00-\u9FFF\u3040-\u309F\u30A0-\u30FF\u31F0-\u31FF\u3000-\u303F]', '', text)

    # 불필요한 공백 제거
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text


In [12]:
df['review'] = df['review'].apply(preprocess_text)

In [13]:
# 'review' 열에서 null 또는 빈 값을 제거
df = df[df['review'].notnull() & (df['review'].str.strip() != '')]

In [14]:
print(df.shape)

(29146, 21)


In [8]:
for text in df['review']:
    print(text)

너무너무 재미있어용
친구랑 하면 재미있음 근데 점점 재미없어짐
캐릭터가 움직이는 게 귀엽습니다 한 명만 통과해도 세이브가 되어 게임을 클리어하는 재미가 떨어집니다 친구와 함께 협동하기 좋습니다
짱박아두면 한번씩 스테이지 업글해줌 개꿀
확실히 재미는 있어용 하지만 3d멀미 심한 사람은 지옥을 겪을 수 있으니 조심하세요 나도 알고싶지 않았어요
개재밌습니다 꼭 하세요 친구들과 함께
친구들과 함께하기 좋은 게임
여러명이서 같이 해야 꿀잼인 게임
아니분명잡았다고왜놓냐고마우스누르고있는데놓지말라고일어서라고왜멋대로쓰러지는데
말랑말랑해요
친구랑하면 너무 재밌어요
몰루
개재밌음
존나 재뮜어요우 저는 로스엔젤레스에서 게임울 하고있어요우
걍 존나 재밌다 당장 구매해라
친구랑 하면 이거만큼 재밌는 게임이 없음 고인물 하나 끼고하면 재밌음
할 거 없을때 친구들이랑 하기 좋아요
캐릭터들 존나 멍청하게 움직임 근데 그게 재밌음
85640
공방들어가면 킥함 모자란 친구들이 많은겜 친구랑만 하면 재미는있음
오
머리끄댕이 잡지마
내 기준에선 별로였음 컨트롤도 어려워서 제대로 못하겠고 재미는 있긴 한데 친구들이 알아서 다 하니까 음 그저 그럼
주기적으로 한번씩 해야하는겜 근데 맨날 엉덩이 자랑만 하다가 끈다
친구들이랑 하면 이만큼 재밌는 겜이 따로 없음
일단 파쿠르 느낌이라서 좋음 맵도 다양하고 캐릭도 창작마당에서 모드 불러와서 편하게 할 수 있어서 다 좋음 근데 크리스마스 맵에서 플레이어들이 자꾸 폭죽 쏴서 괴롭힘 밴 기능이 있긴 한데 계속 들어옴 그것만 고쳐주면 완전 갓겜임
같이 했다
흐물흐물
처음으로 스트레스 하나도 안받으면서 했던게임
양손 남의 머리잡고 화면 위아래로 흔들면 석 마 딕 시전가능
이거만큼 뇌빼고 하기 좋은겜이 어디있을까요
너무어려움 조작이
흐물흐물해
이 것이 즐겜이다 희망편
귀여움 조금 지루함
재밌음
친구 없으면 하지마라 혼자해보면 알거다 ㅅ1ㅂr
5천원 이하면 한다
재미있ㅇ름
머리좀 그만 잡어 ㅅ1ㅂ
재밌음
체고에용 가 리 치 킨
ദദᵔ ᵔ
평소에도 못했는 손잡는걸 이게임

## 불용어 제거

In [15]:
# 불용어 파일 로드
with open('stopwords.word.txt', 'r', encoding='utf-8') as file:
    stopwords = file.read().splitlines()

stopwords_set = set(stopwords)

def remove_stopwords_from_text(text):
    # 텍스트를 단어 단위로 나누기
    words = text.split()
    
    # 불용어가 아닌 단어만 선택
    filtered_words = [word for word in words if word not in stopwords_set]
    
    # 필터링된 단어들을 다시 하나의 문자열로 합침
    return ' '.join(filtered_words)

In [16]:
df['review'] = df['review'].apply(remove_stopwords_from_text)

In [11]:
for text in df['review']:
    print(text)

너무너무 재미있어용
친구랑 하면 재미있음 근데 점점 재미없어짐
캐릭터가 움직이는 게 귀엽습니다 한 명만 통과해도 세이브가 되어 게임을 클리어하는 재미가 떨어집니다 친구와 협동하기 좋습니다
짱박아두면 한번씩 스테이지 업글해줌 개꿀
확실히 재미는 있어용 3d멀미 심한 사람은 지옥을 겪을 수 있으니 조심하세요 나도 알고싶지 않았어요
개재밌습니다 꼭 하세요 친구들과
친구들과 함께하기 좋은 게임
여러명이서 해야 꿀잼인 게임
아니분명잡았다고왜놓냐고마우스누르고있는데놓지말라고일어서라고왜멋대로쓰러지는데
말랑말랑해요
친구랑하면 너무 재밌어요
몰루
개재밌음
존나 재뮜어요우 저는 로스엔젤레스에서 게임울 하고있어요우
걍 존나 재밌다 구매해라
친구랑 하면 이거만큼 재밌는 게임이 없음 고인물 끼고하면 재밌음
할 거 없을때 친구들이랑 하기 좋아요
캐릭터들 존나 멍청하게 움직임 근데 그게 재밌음
85640
공방들어가면 킥함 모자란 친구들이 많은겜 친구랑만 하면 재미는있음

머리끄댕이 잡지마
내 기준에선 별로였음 컨트롤도 어려워서 제대로 못하겠고 재미는 있긴 친구들이 알아서 다 하니까 음
주기적으로 한번씩 해야하는겜 근데 맨날 엉덩이 자랑만 하다가 끈다
친구들이랑 하면 재밌는 겜이 따로 없음
파쿠르 느낌이라서 좋음 맵도 다양하고 캐릭도 창작마당에서 모드 불러와서 편하게 할 수 있어서 다 좋음 근데 크리스마스 맵에서 플레이어들이 자꾸 폭죽 쏴서 괴롭힘 밴 기능이 있긴 계속 들어옴 그것만 고쳐주면 완전 갓겜임
했다
흐물흐물
처음으로 스트레스 하나도 안받으면서 했던게임
양손 남의 머리잡고 화면 위아래로 흔들면 석 마 딕 시전가능
이거만큼 뇌빼고 하기 좋은겜이 어디있을까요
너무어려움 조작이
흐물흐물해
것이 즐겜이다 희망편
귀여움 지루함
재밌음
친구 없으면 혼자해보면 알거다 ㅅ1ㅂr
5천원 이하면 한다
재미있ㅇ름
머리좀 그만 잡어 ㅅ1ㅂ
재밌음
체고에용 리 치 킨
ദദᵔ ᵔ
평소에도 못했는 손잡는걸 이게임에서 하고있음ㅇㅇ
협동게임 협동게임 최고라고 생각합니다 재밌었어요
개판 오분전
머리 끄댕이 잡는걸

In [17]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
df['review'] = df['review'].apply(lambda text: ' '.join([word for word in text.split() if word not in stop_words]))

In [13]:
for text in df['review']:
    print(text)

너무너무 재미있어용
친구랑 하면 재미있음 근데 점점 재미없어짐
캐릭터가 움직이는 게 귀엽습니다 한 명만 통과해도 세이브가 되어 게임을 클리어하는 재미가 떨어집니다 친구와 협동하기 좋습니다
짱박아두면 한번씩 스테이지 업글해줌 개꿀
확실히 재미는 있어용 3d멀미 심한 사람은 지옥을 겪을 수 있으니 조심하세요 나도 알고싶지 않았어요
개재밌습니다 꼭 하세요 친구들과
친구들과 함께하기 좋은 게임
여러명이서 해야 꿀잼인 게임
아니분명잡았다고왜놓냐고마우스누르고있는데놓지말라고일어서라고왜멋대로쓰러지는데
말랑말랑해요
친구랑하면 너무 재밌어요
몰루
개재밌음
존나 재뮜어요우 저는 로스엔젤레스에서 게임울 하고있어요우
걍 존나 재밌다 구매해라
친구랑 하면 이거만큼 재밌는 게임이 없음 고인물 끼고하면 재밌음
할 거 없을때 친구들이랑 하기 좋아요
캐릭터들 존나 멍청하게 움직임 근데 그게 재밌음
85640
공방들어가면 킥함 모자란 친구들이 많은겜 친구랑만 하면 재미는있음

머리끄댕이 잡지마
내 기준에선 별로였음 컨트롤도 어려워서 제대로 못하겠고 재미는 있긴 친구들이 알아서 다 하니까 음
주기적으로 한번씩 해야하는겜 근데 맨날 엉덩이 자랑만 하다가 끈다
친구들이랑 하면 재밌는 겜이 따로 없음
파쿠르 느낌이라서 좋음 맵도 다양하고 캐릭도 창작마당에서 모드 불러와서 편하게 할 수 있어서 다 좋음 근데 크리스마스 맵에서 플레이어들이 자꾸 폭죽 쏴서 괴롭힘 밴 기능이 있긴 계속 들어옴 그것만 고쳐주면 완전 갓겜임
했다
흐물흐물
처음으로 스트레스 하나도 안받으면서 했던게임
양손 남의 머리잡고 화면 위아래로 흔들면 석 마 딕 시전가능
이거만큼 뇌빼고 하기 좋은겜이 어디있을까요
너무어려움 조작이
흐물흐물해
것이 즐겜이다 희망편
귀여움 지루함
재밌음
친구 없으면 혼자해보면 알거다 ㅅ1ㅂr
5천원 이하면 한다
재미있ㅇ름
머리좀 그만 잡어 ㅅ1ㅂ
재밌음
체고에용 리 치 킨
ദദᵔ ᵔ
평소에도 못했는 손잡는걸 이게임에서 하고있음ㅇㅇ
협동게임 협동게임 최고라고 생각합니다 재밌었어요
개판 오분전
머리 끄댕이 잡는걸

In [18]:
df.to_parquet('output_cleaned.parquet')

## 감성 분석 시 필요 없는 필드 제거

In [19]:
df.columns

Index(['recommendationid', 'review', 'timestamp_created', 'timestamp_updated',
       'voted_up', 'votes_up', 'votes_funny', 'weighted_vote_score',
       'comment_count', 'steam_purchase', 'received_for_free',
       'written_during_early_access', 'primarily_steam_deck', 'steamid',
       'num_games_owned', 'num_reviews', 'playtime_forever',
       'playtime_last_two_weeks', 'playtime_at_review', 'last_played',
       'deck_playtime_at_review'],
      dtype='object')

weighted_vote_score, last_played, votes_up, written_during_early_access, steam_purchase

In [20]:
df.drop(columns=['recommendationid', 'timestamp_created', 'timestamp_updated',
    'votes_funny',
    'comment_count', 'received_for_free',
    'primarily_steam_deck', 'steamid',
    'num_games_owned', 'num_reviews', 'playtime_forever',
    'playtime_last_two_weeks', 'playtime_at_review',
    'deck_playtime_at_review'], inplace=True)

In [21]:
df.columns

Index(['review', 'voted_up', 'votes_up', 'weighted_vote_score',
       'steam_purchase', 'written_during_early_access', 'last_played'],
      dtype='object')

In [22]:
df.to_parquet('output_run.parquet')